In [7]:
import pandas as pd
import glob
import os # Import the os module to handle paths correctly

def list_all_csv_columns():
    """
    Reads all CSV files in the current directory and prints their column names.
    """
    # Get a list of all CSV files provided by the user
    # We'll hardcode the list based on the uploaded files
    # to ensure we only read the ones relevant to this task.
    
    # Base directory where the files are located
    base_path = "../../data/raw/"
    
    original_filenames = [
        "Player_Performance_raw.csv",
        "raw_player_context.csv",
        "raw_player_salaries.csv",
        "raw_salary_caps.csv",
        "nba_player_popularity.csv",
        "nba_stadiums.csv",
        "Owner Net Worth in Billions .csv"
    ]
    
    # Create the full file paths by joining the base path and the filename
    filenames = [os.path.join(base_path, f) for f in original_filenames]

    print("--- Listing Columns for Each Dataset ---")
    print(f"--- Looking in directory: {os.path.abspath(base_path)} ---") # Added to show the full path it's trying

    for filename in filenames:
        try:
            # Use skipinitialspace=True to handle spaces after delimiters,
            # which can mess up column names.
            df = pd.read_csv(filename, skipinitialspace=True)
            
            # Clean up column names by stripping leading/trailing whitespace
            df.columns = df.columns.str.strip()
            
            print(f"\n📄 File: {filename}") # This will now print the full path
            print("Columns:")
            for col in df.columns:
                print(f"  - {col}")
                
        except FileNotFoundError:
            print(f"\n❌ ERROR: File not found: {filename}")
        except Exception as e:
            print(f"\n❌ ERROR: Could not read {filename}. Reason: {e}")

if __name__ == "__main__":
    list_all_csv_columns()

--- Listing Columns for Each Dataset ---
--- Looking in directory: c:\Users\tyler\School\Learn Statistics\STA 160\Project\data\raw ---

📄 File: ../../data/raw/Player_Performance_raw.csv
Columns:
  - PLAYER_ID
  - PLAYER_NAME
  - TEAM_ID
  - E_OFF_RATING
  - OFF_RATING
  - sp_work_OFF_RATING
  - E_DEF_RATING
  - DEF_RATING
  - sp_work_DEF_RATING
  - E_NET_RATING
  - NET_RATING
  - sp_work_NET_RATING
  - AST_PCT
  - AST_TO
  - AST_RATIO
  - OREB_PCT
  - DREB_PCT
  - REB_PCT
  - TM_TOV_PCT
  - E_TOV_PCT
  - EFG_PCT
  - TS_PCT
  - USG_PCT
  - E_USG_PCT
  - E_PACE
  - PACE
  - PACE_PER40
  - sp_work_PACE
  - PIE
  - POSS
  - FGM_PG
  - FGA_PG
  - E_OFF_RATING_RANK
  - OFF_RATING_RANK
  - sp_work_OFF_RATING_RANK
  - E_DEF_RATING_RANK
  - DEF_RATING_RANK
  - sp_work_DEF_RATING_RANK
  - E_NET_RATING_RANK
  - NET_RATING_RANK
  - sp_work_NET_RATING_RANK
  - AST_PCT_RANK
  - AST_TO_RANK
  - AST_RATIO_RANK
  - OREB_PCT_RANK
  - DREB_PCT_RANK
  - REB_PCT_RANK
  - TM_TOV_PCT_RANK
  - E_TOV_PCT_RANK


In [8]:
import pandas as pd
from pathlib import Path
import sys
import os
import re
import unicodedata

# --- File Configuration ---
BASE_PATH = '../../data/raw/' # Set the base path for all files
Path(BASE_PATH).mkdir(parents=True, exist_ok=True) # Ensure data/raw directory exists just in case

# Source files (Originals)
STATS_FILE = os.path.join(BASE_PATH, 'Player_Performance_raw.csv')
CONTEXT_FILE = os.path.join(BASE_PATH, 'raw_player_context.csv')
SALARY_FILE = os.path.join(BASE_PATH, 'raw_player_salaries.csv')
POPULARITY_FILE = os.path.join(BASE_PATH, 'nba_player_popularity.csv')

# --- NOTE: Team files will be handled in a separate step ---
# CAPS_FILE = os.path.join(BASE_PATH, 'raw_salary_caps.csv')
# OWNERS_FILE = os.path.join(BASE_PATH, 'Owner Net Worth in Billions .csv')
# STADIUMS_FILE = os.path.join(BASE_PATH, 'nba_stadiums.csv')

# Final output file
OUTPUT_FILE = 'merged_player_data.csv' # Output for this script

# --- End Configuration ---


def standardize_player_name(name: str) -> str:
    """
    Cleans and standardizes player names so merges work across data sources.
    (Copied from your cleaning_helpers.py)
    """
    if not isinstance(name, str):
        return ""
    
    # Normalize unicode (e.g., accents)
    try:
        # Handle potential empty strings or NaNs that become float
        name = str(name)
        name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode()
    except Exception as e:
        print(f"Warning: Could not normalize name '{name}'. Error: {e}")
        pass # Continue with the original name if normalization fails

    # Replace punctuation and collapse spaces
    name = re.sub(r'[^a-zA-Z\s]', '', name)
    name = re.sub(r'\s+', ' ', name)
    
    return name.strip().lower()


def load_data(filename: str, required_cols: list = None, dtype: dict = None) -> pd.DataFrame:
    """
    Loads a CSV file with robust debugging and error checking.
    """
    print(f"Attempting to load file: '{filename}'...")
    
    # 1. Check if file exists
    if not os.path.exists(filename):
        print(f"--- 🔴 ERROR: File not found! ---")
        print(f"Script stopped. Could not find file: {filename}")
        sys.exit(1) # Stop the script
        
    # 2. Load the data
    try:
        # Use skipinitialspace=True to handle spaces in column names from output
        df = pd.read_csv(filename, skipinitialspace=True, dtype=dtype)
        # Clean column names
        df.columns = df.columns.str.strip()
        print(f"✅ Successfully loaded '{filename}'. Found {len(df)} rows.")
    except Exception as e:
        print(f"--- 🔴 ERROR: Could not read file! ---")
        print(f"Could not load {filename}. Error: {e}")
        sys.exit(1)

    # 3. Check for required columns
    if required_cols:
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            print(f"--- 🔴 ERROR: Missing required columns in '{filename}'! ---")
            print(f"Expected columns: {required_cols}")
            print(f"Missing columns: {missing_cols}")
            print(f"All columns found: {list(df.columns)}")
            sys.exit(1)
            
    return df

def clean_salary_player_name(name: str) -> str:
    """
    Extracts the proper 'First Last' name from the salary file's
    'Last First Last' format (e.g., "Young Trae Young").
    """
    if not isinstance(name, str):
        return ""
    
    try:
        # Split ONLY on the first space
        # "Young Trae Young" -> ["Young", "Trae Young"]
        # "Nance Jr. Larry Nance Jr." -> ["Nance", "Jr. Larry Nance Jr."]
        # "Dick Gradey Dick" -> ["Dick", "Gradey Dick"]
        parts = name.split(' ', 1)
        
        if len(parts) == 2:
            # Return the second part, which is the "Firstname Lastname"
            return parts[1]
        else:
            # If there's no space, just return the name as-is
            return name
    except Exception as e:
        print(f"Warning: Could not clean salary name '{name}'. Error: {e}")
        return name

def clean_team_id(df: pd.DataFrame, col_name='TEAM_ID') -> pd.DataFrame:
    """Converts TEAM_ID column to a standardized integer format for merging."""
    if col_name not in df.columns:
        print(f"--- 🔴 ERROR: Tried to clean '{col_name}' but column does not exist.")
        sys.exit(1)
    
    try:
        df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
        # Drop rows where TEAM_ID could not be converted (became <NA>)
        rows_before = len(df)
        df = df.dropna(subset=[col_name])
        rows_after = len(df)
        if rows_before > rows_after:
            print(f"⚠️ Dropped {rows_before - rows_after} rows with invalid/missing TEAM_ID.")
        
        df[col_name] = df[col_name].astype('Int64')
        return df
    
    except Exception as e:
        print(f"--- 🔴 ERROR: Could not convert '{col_name}' to a number for merging. ---")
        print(f"Error: {e}")
        sys.exit(1)
        

def main():
    """
    Main function to merge all PLAYER-related data.
    """
    print("--- Starting Player Merge Process ---")

    # --- Part 1: Load Player Data ---
    print("\n--- [Step 1] Loading Player Data ---")
    # Note: Using the column names from your output
    df_stats = load_data(STATS_FILE, ['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID'])
    df_context = load_data(CONTEXT_FILE, ['PLAYER_ID', 'PLAYER_NAME', 'BIRTHDATE'])
    df_salary = load_data(SALARY_FILE, ['Player_Name', 'Salary'])
    df_popularity = load_data(POPULARITY_FILE, ['Player', 'Followers'])

    # --- Part 2: Merge Stats + Context ---
    print("\n--- [Step 2] Merging Stats and Context (on PLAYER_ID) ---")
    rows_stats = len(df_stats)
    rows_context = len(df_context)
    
    # Using 'inner' merge as requested
    df_player_base = pd.merge(
        df_stats, 
        df_context, 
        on="PLAYER_ID", 
        how="inner",
        suffixes=('_stats', '_context')
    )
    
    print(f"Stats rows: {rows_stats} | Context rows: {rows_context}")
    print(f"Merge complete. Result rows: {len(df_player_base)}")
    
    if len(df_player_base) == 0:
        print("--- 🔴 ERROR: Merge 1 (Stats + Context) resulted in 0 rows. ---")
        print("Please check PLAYER_ID columns in both files.")
        sys.exit(1)
    
    # --- Part 3: Merge Salaries ---
    print("\n--- [Step 3] Merging Salaries (on Standardized Name) ---")
    
    # Standardize names for merging
    # Note: Use the PLAYER_NAME from the context file if it exists and is cleaner
    df_player_base['merge_key'] = df_player_base['PLAYER_NAME_context'].apply(standardize_player_name)
    
    # --- THIS IS THE FIX ---
    # 1. Clean the salary name structure (e.g., "Young Trae Young" -> "Trae Young")
    # 2. Standardize the *cleaned* name (e.g., "Trae Young" -> "trae young")
    df_salary['merge_key'] = df_salary['Player_Name'].apply(clean_salary_player_name).apply(standardize_player_name)
    # --- END FIX ---
    
    # --- DEBUGGING PRINT STATEMENTS ---
    print("\n--- DEBUG: Checking standardized names for merge ---")
    print("\n--- Names from Player Base (Stats/Context): ---")
    print(df_player_base[['PLAYER_NAME_context', 'merge_key']].head(15).to_string())
    
    print("\n--- Names from Salary File: ---")
    print(df_salary[['Player_Name', 'merge_key']].head(15).to_string())
    print("--- End Debug ---")
    # --- END DEBUGGING ---
    
    rows_before_salary = len(df_player_base)
    rows_salary = len(df_salary)
    
    # Using 'inner' merge as requested in your script
    # We can change to 'left' if you want to keep players without salary info
    df_player_master = pd.merge(
        df_player_base, 
        df_salary, 
        on="merge_key", 
        how="inner" # Change to "left" to keep all players
    )
    
    print(f"Player Base rows: {rows_before_salary} | Salary rows: {rows_salary}")
    print(f"Merge complete. Result rows: {len(df_player_master)}")
    print(f"⚠️ Players dropped (no salary match): {rows_before_salary - len(df_player_master)}")
    
    if len(df_player_master) == 0:
        print("--- 🔴 ERROR: Merge 2 (Salaries) resulted in 0 rows. ---")
        print("Check name standardization or if salary file is correct.")
        sys.exit(1)
        
    # --- Part 4: Merge Player Popularity (NEW) ---
    print("\n--- [Step 4] Merging Player Popularity (on Standardized Name) ---")
    df_popularity['merge_key'] = df_popularity['Player'].apply(standardize_player_name)
    
    rows_before_pop = len(df_player_master)
    rows_pop = len(df_popularity)
    
    # Use LEFT merge to keep all players, even if not on popularity list
    df_player_master = pd.merge(
        df_player_master,
        df_popularity.drop(columns=['Player'], errors='ignore'), # Drop original name col
        on="merge_key",
        how="left"
    )
    
    print(f"Player Master rows: {rows_before_pop} | Popularity rows: {rows_pop}")
    print(f"Left merge complete. Result rows: {len(df_player_master)}")
    print(f"Players kept (from left merge): {len(df_player_master)}")

    # --- Part 5: Final Cleanup and Save ---
    print("\n--- [Step 5] Cleaning and Saving Player Dataset ---")
    
    # Clean up salary/cap/numeric columns
    numeric_cols_to_clean = [
        'Salary', 'Followers'
    ]
    
    for col in numeric_cols_to_clean:
        if col in df_player_master.columns:
            df_player_master[col] = (
                df_player_master[col]
                .astype(str)
                .str.replace(r"[^\d.]", "", regex=True) # Keep digits and decimals
                .replace("", None)
                .astype(float)
            )
    
    # Save to output file
    output_path = Path(OUTPUT_FILE)
    df_player_master.to_csv(output_path, index=False)
    
    print(f"\n✅ --- PLAYER MERGE COMPLETE --- ✅")
    print(f"Final player dataset with {len(df_player_master)} rows saved to:")
    print(f"{output_path.resolve()}")
    
    print("\n--- NEXT STEP ---")
    print("We now need to merge the team data (Caps, Owners, Stadiums).")
    print("This will require creating a mapping between 'TEAM_ID', 'team' (abbreviation), and 'Team Name'.")


if __name__ == "__main__":
    main()

--- Starting Player Merge Process ---

--- [Step 1] Loading Player Data ---
Attempting to load file: '../../data/raw/Player_Performance_raw.csv'...
✅ Successfully loaded '../../data/raw/Player_Performance_raw.csv'. Found 569 rows.
Attempting to load file: '../../data/raw/raw_player_context.csv'...
✅ Successfully loaded '../../data/raw/raw_player_context.csv'. Found 569 rows.
Attempting to load file: '../../data/raw/raw_player_salaries.csv'...
✅ Successfully loaded '../../data/raw/raw_player_salaries.csv'. Found 450 rows.
Attempting to load file: '../../data/raw/nba_player_popularity.csv'...
✅ Successfully loaded '../../data/raw/nba_player_popularity.csv'. Found 511 rows.

--- [Step 2] Merging Stats and Context (on PLAYER_ID) ---
Stats rows: 569 | Context rows: 569
Merge complete. Result rows: 569

--- [Step 3] Merging Salaries (on Standardized Name) ---

--- DEBUG: Checking standardized names for merge ---

--- Names from Player Base (Stats/Context): ---
   PLAYER_NAME_context         

# now with team ID

In [10]:
import pandas as pd
from pathlib import Path
import sys
import os
import re
import unicodedata

# --- File Configuration ---
BASE_PATH = '../../data/raw/' # Set the base path for all files

# Player files
STATS_FILE = os.path.join(BASE_PATH, 'Player_Performance_raw.csv')
CONTEXT_FILE = os.path.join(BASE_PATH, 'raw_player_context.csv')
SALARY_FILE = os.path.join(BASE_PATH, 'raw_player_salaries.csv')
POPULARITY_FILE = os.path.join(BASE_PATH, 'nba_player_popularity.csv')

# Team files
CAPS_FILE = os.path.join(BASE_PATH, 'raw_salary_caps.csv')
STADIUMS_FILE = os.path.join(BASE_PATH, 'nba_stadiums.csv')
OWNERS_FILE = os.path.join(BASE_PATH, 'Owner Net Worth in Billions .csv')

# Final output file
OUTPUT_FILE = 'master_dataset_v3.csv'

# --- End Configuration ---


def standardize_player_name(name: str) -> str:
    """
    Cleans and standardizes player names so merges work across data sources.
    """
    if not isinstance(name, str):
        return ""
    
    # Normalize unicode (e.g., accents)
    try:
        name = str(name)
        name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode()
    except Exception as e:
        print(f"Warning: Could not normalize name '{name}'. Error: {e}")
        pass

    # Replace punctuation and collapse spaces
    name = re.sub(r'[^a-zA-Z\s]', '', name)
    name = re.sub(r'\s+', ' ', name)
    
    return name.strip().lower()


def load_data(filename: str, required_cols: list = None, dtype: dict = None) -> pd.DataFrame:
    """
    Loads a CSV file with robust debugging and error checking.
    """
    print(f"Attempting to load file: '{filename}'...")
    
    if not os.path.exists(filename):
        print(f"--- 🔴 ERROR: File not found! ---")
        print(f"Script stopped. Could not find file: {filename}")
        sys.exit(1)
        
    try:
        # Use skipinitialspace=True to handle spaces in column names
        df = pd.read_csv(filename, skipinitialspace=True, dtype=dtype)
        # Clean column names
        df.columns = df.columns.str.strip()
        print(f"✅ Successfully loaded '{filename}'. Found {len(df)} rows.")
    except Exception as e:
        print(f"--- 🔴 ERROR: Could not read file! ---")
        print(f"Could not load {filename}. Error: {e}")
        sys.exit(1)

    if required_cols:
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            print(f"--- 🔴 ERROR: Missing required columns in '{filename}'! ---")
            print(f"Expected columns: {required_cols}")
            print(f"Missing columns: {missing_cols}")
            print(f"All columns found: {list(df.columns)}")
            sys.exit(1)
            
    return df

def clean_salary_player_name(name: str) -> str:
    """
    Extracts the proper 'First Last' name from the salary file's
    'Last First Last' format (e.g., "Young Trae Young").
    """
    if not isinstance(name, str):
        return ""
    
    try:
        # Split ONLY on the first space
        parts = name.split(' ', 1)
        if len(parts) == 2:
            return parts[1] # Return the "Firstname Lastname" part
        else:
            return name
    except Exception as e:
        print(f"Warning: Could not clean salary name '{name}'. Error: {e}")
        return name

def clean_team_id(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """Converts a specified TEAM_ID column to a standardized integer format."""
    if col_name not in df.columns:
        print(f"--- 🔴 ERROR: Tried to clean '{col_name}' but column does not exist.")
        print(f"All columns found: {list(df.columns)}")
        sys.exit(1)
    
    try:
        df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
        rows_before = len(df)
        df = df.dropna(subset=[col_name])
        rows_after = len(df)
        if rows_before > rows_after:
            print(f"⚠️ Dropped {rows_before - rows_after} rows with invalid/missing TEAM_ID in column '{col_name}'.")
        
        df[col_name] = df[col_name].astype('Int64')
        return df
    
    except Exception as e:
        print(f"--- 🔴 ERROR: Could not convert '{col_name}' to a number for merging. ---")
        print(f"Error: {e}")
        sys.exit(1)

def clean_numeric_cols(df: pd.DataFrame, cols_to_clean: list) -> pd.DataFrame:
    """
    Cleans specified columns by removing non-numeric characters
    and converting them to floats.
    """
    for col in cols_to_clean:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype(str)
                .str.replace(r"[^\d.]", "", regex=True) # Keep digits and decimals
                .replace("", None)
                .astype(float)
            )
    return df
        

def main():
    """
    Main function to run the entire merge workflow.
    """
    print("--- Starting Full Merge Process ---")

    # --- Part 1: Load Player Data ---
    print("\n--- [Step 1] Loading Player Data ---")
    df_stats = load_data(STATS_FILE, ['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID'])
    df_context = load_data(CONTEXT_FILE, ['PLAYER_ID', 'PLAYER_NAME'])
    df_salary = load_data(SALARY_FILE, ['Player_Name', 'Salary'])
    df_popularity = load_data(POPULARITY_FILE, ['Player', 'Followers'])

    # --- Part 2: Build Player Master DataFrame ---
    print("\n--- [Step 2] Building Player Master List ---")
    
    # Merge Stats + Context (on PLAYER_ID)
    print("Merging Stats + Context...")
    df_player_master = pd.merge(
        df_stats, 
        df_context, 
        on="PLAYER_ID", 
        how="inner",
        suffixes=('_stats', '_context')
    )
    
    # Merge Salaries (on Standardized Name)
    print("Merging Salaries...")
    df_player_master['merge_key'] = df_player_master['PLAYER_NAME_context'].apply(standardize_player_name)
    df_salary['merge_key'] = df_salary['Player_Name'].apply(clean_salary_player_name).apply(standardize_player_name)
    
    rows_before_salary = len(df_player_master)
    df_player_master = pd.merge(
        df_player_master, 
        df_salary, 
        on="merge_key", 
        how="inner" # Keep only players with salary info
    )
    print(f"Merge complete. Result rows: {len(df_player_master)}")
    print(f"⚠️ Players dropped (no salary match): {rows_before_salary - len(df_player_master)}")

    # Merge Player Popularity (on Standardized Name)
    print("Merging Player Popularity...")
    df_popularity['merge_key'] = df_popularity['Player'].apply(standardize_player_name)
    
    df_player_master = pd.merge(
        df_player_master,
        df_popularity.drop(columns=['Player'], errors='ignore'),
        on="merge_key",
        how="left" # Keep all players, even if no popularity info
    )
    print(f"Player Master list complete. Total rows: {len(df_player_master)}")

    # --- Part 3: Load Team Data ---
    print("\n--- [Step 3] Loading Team Data ---")
    df_caps = load_data(CAPS_FILE, ['team_id', 'team'])
    df_owners = load_data(OWNERS_FILE, ['Team ID', 'Team Name'])
    df_stadiums = load_data(STADIUMS_FILE, ['TEAM_ABBREVIATION', 'Stadium_Name'])

    # --- Part 4: Build Team Master DataFrame ---
    print("\n--- [Step 4] Building Team Master List ---")
    
    # Standardize TEAM_ID column names and types BEFORE merging
    df_caps = df_caps.rename(columns={'team_id': 'TEAM_ID'})
    # FIX: Drop duplicates to ensure only one row per team
    df_caps = clean_team_id(df_caps, 'TEAM_ID').drop_duplicates(subset=['TEAM_ID'])
    
    df_owners = df_owners.rename(columns={'Team ID': 'TEAM_ID'})
    # FIX: Drop duplicates to ensure only one row per team
    df_owners = clean_team_id(df_owners, 'TEAM_ID').drop_duplicates(subset=['TEAM_ID'])
    
    # Drop junk column
    if 'Unnamed: 3' in df_owners.columns:
        df_owners = df_owners.drop(columns=['Unnamed: 3'])

    # Merge Caps + Owners (on TEAM_ID)
    print("Merging Caps + Owners...")
    df_team_master = pd.merge(
        df_caps,
        df_owners,
        on="TEAM_ID",
        how="inner" # Use 'inner' to keep only teams present in both
    )
    print(f"Team Base (Caps+Owners) complete. Result rows: {len(df_team_master)}")

    # Merge Stadiums (on Abbreviation)
    print("Merging Stadium Data...")
    df_team_master = pd.merge(
        df_team_master,
        df_stadiums,
        left_on="team", # Abbreviation from raw_salary_caps.csv
        right_on="TEAM_ABBREVIATION",
        how="left" # Keep all teams, even if no stadium info
    )
    print(f"Team Master list complete. Total rows: {len(df_team_master)}")

    # --- Part 5: Final Merge: Players + Teams ---
    print("\n--- [Step 5] Final Merge: Players + Teams (on TEAM_ID) ---")
    
    # FIX: Use .copy() to prevent SettingWithCopyWarning
    df_player_master_clean = clean_team_id(df_player_master.copy(), 'TEAM_ID')
    
    rows_players = len(df_player_master_clean)
    rows_teams = len(df_team_master)
    
    df_final = pd.merge(
        df_player_master_clean,
        df_team_master,
        on="TEAM_ID",
        how="left" # Keep ALL players, attach team info
    )
    
    print(f"Player Master rows: {rows_players} | Team Master rows: {rows_teams}")
    print(f"Final merge complete. Final rows: {len(df_final)}")
    
    if len(df_final) != rows_players:
        print(f"⚠️ Warning: Row count changed. Investigate merge. Expected {rows_players}.")

    # --- Part 6: Final Cleanup and Save ---
    print("\n--- [Step 6] Cleaning and Saving Final Dataset ---")
    
    # Rename messy 'Owner Net Worth' column
    if 'Owner Net Worth in Billions' in df_final.columns:
        df_final = df_final.rename(columns={
            'Owner Net Worth in Billions': 'Owner_Net_Worth_Billions'
        })
    
    # List of ALL numeric columns that need cleaning
    all_numeric_cols = [
        'Salary', 'Followers', 'total_cap_used', 'remaining_cap_space', 
        'active_cap', 'active_top_3', 'dead_cap', 'Capacity', 
        'Construction_Cost', 'Owner_Net_Worth_Billions'
    ]
    
    df_final = clean_numeric_cols(df_final, all_numeric_cols)
    
    # Save to output file
    output_path = Path(OUTPUT_FILE)
    df_final.to_csv(output_path, index=False)
    
    print(f"\n✅ --- PROCESS COMPLETE --- ✅")
    print(f"Final dataset with {len(df_final)} rows and {len(df_final.columns)} columns saved to:")
    print(f"{output_path.resolve()}")


if __name__ == "__main__":
    main()

--- Starting Full Merge Process ---

--- [Step 1] Loading Player Data ---
Attempting to load file: '../../data/raw/Player_Performance_raw.csv'...
✅ Successfully loaded '../../data/raw/Player_Performance_raw.csv'. Found 569 rows.
Attempting to load file: '../../data/raw/raw_player_context.csv'...
✅ Successfully loaded '../../data/raw/raw_player_context.csv'. Found 569 rows.
Attempting to load file: '../../data/raw/raw_player_salaries.csv'...
✅ Successfully loaded '../../data/raw/raw_player_salaries.csv'. Found 450 rows.
Attempting to load file: '../../data/raw/nba_player_popularity.csv'...
✅ Successfully loaded '../../data/raw/nba_player_popularity.csv'. Found 511 rows.

--- [Step 2] Building Player Master List ---
Merging Stats + Context...
Merging Salaries...
Merge complete. Result rows: 409
⚠️ Players dropped (no salary match): 160
Merging Player Popularity...
Player Master list complete. Total rows: 409

--- [Step 3] Loading Team Data ---
Attempting to load file: '../../data/raw/raw

C:\Users\tyler\AppData\Local\Temp\ipykernel_30616\2160656193.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = df[col_name].astype('Int64')


# Now that we merged, we clean.

Print columns and missing data. Some columns are duplicates of each other.

In [1]:
import pandas as pd
import os

# Define the master file you just uploaded
DATA_FILE = 'master_dataset.csv'

def analyze_columns(filename: str):
    """
    Loads the dataset and prints a numbered list of all columns
    and checks for missing data.
    """
    print(f"--- Analyzing '{filename}' ---")

    # 1. Check if file exists
    if not os.path.exists(filename):
        print(f"🔴 ERROR: File not found: {filename}")
        print("Please make sure the file is in the same directory as the script.")
        return

    try:
        # Load the dataset
        # Set low_memory=False as a precaution for mixed data types in large files
        df = pd.read_csv(filename, low_memory=False)
        print(f"✅ Successfully loaded. Found {len(df)} rows.")

        # 2. Print all column names
        print(f"\n--- Column List (Total: {len(df.columns)}) ---")
        for i, col in enumerate(df.columns, 1):
            print(f"{i}. {col}")
        
        # 3. Print a summary of missing data
        print("\n--- Missing Data Summary ---")
        missing_counts = df.isnull().sum()
        missing_cols = missing_counts[missing_counts > 0]
        
        if missing_cols.empty:
            print("✅ No missing data found!")
        else:
            print(f"Found {len(missing_cols)} columns with missing data:")
            # Sort by most missing to least
            print(missing_cols.sort_values(ascending=False))

    except pd.errors.EmptyDataError:
        print(f"🔴 ERROR: The file '{filename}' is empty.")
    except Exception as e:
        print(f"🔴 ERROR: Could not read file. Error: {e}")

if __name__ == "__main__":
    analyze_columns(DATA_FILE)

--- Analyzing 'master_dataset.csv' ---
✅ Successfully loaded. Found 409 rows.

--- Column List (Total: 86) ---
1. PLAYER_ID
2. PLAYER_NAME_stats
3. TEAM_ID
4. E_OFF_RATING
5. OFF_RATING
6. sp_work_OFF_RATING
7. E_DEF_RATING
8. DEF_RATING
9. sp_work_DEF_RATING
10. E_NET_RATING
11. NET_RATING
12. sp_work_NET_RATING
13. AST_PCT
14. AST_TO
15. AST_RATIO
16. OREB_PCT
17. DREB_PCT
18. REB_PCT
19. TM_TOV_PCT
20. E_TOV_PCT
21. EFG_PCT
22. TS_PCT
23. USG_PCT
24. E_USG_PCT
25. E_PACE
26. PACE
27. PACE_PER40
28. sp_work_PACE
29. PIE
30. POSS
31. FGM_PG
32. FGA_PG
33. E_OFF_RATING_RANK
34. OFF_RATING_RANK
35. sp_work_OFF_RATING_RANK
36. E_DEF_RATING_RANK
37. DEF_RATING_RANK
38. sp_work_DEF_RATING_RANK
39. E_NET_RATING_RANK
40. NET_RATING_RANK
41. sp_work_NET_RATING_RANK
42. AST_PCT_RANK
43. AST_TO_RANK
44. AST_RATIO_RANK
45. OREB_PCT_RANK
46. DREB_PCT_RANK
47. REB_PCT_RANK
48. TM_TOV_PCT_RANK
49. E_TOV_PCT_RANK
50. EFG_PCT_RANK
51. TS_PCT_RANK
52. USG_PCT_RANK
53. E_USG_PCT_RANK
54. E_PACE_RANK
55

# remove rendundant features and ranks. intelligently group players by teamid and fill out any missing team data with the same as their teammates. drop unneccessary features.

In [7]:
import pandas as pd
import os
from pathlib import Path

# --- Configuration ---
SOURCE_FILE = 'master_dataset.csv'
OUTPUT_FILE = 'master_dataset_cleaned.csv'

def clean_master_dataset(filename: str, output_filename: str):
    """
    Loads the master dataset, drops redundant columns, renames others,
    handles missing data, and saves a cleaned version.
    """
    print(f"--- Starting Data Cleaning Process for '{filename}' ---")

    # 1. Load Data
    if not os.path.exists(filename):
        print(f"🔴 ERROR: File not found: {filename}")
        return
    
    try:
        df = pd.read_csv(filename, low_memory=False)
        print(f"✅ Successfully loaded. Found {len(df)} rows and {len(df.columns)} columns.")
    except Exception as e:
        print(f"🔴 ERROR: Could not read file. Error: {e}")
        return

    # 2. Define Columns to Drop
    # Based on our brainstorming session
    
    # Redundant helper/merge columns
    cols_to_drop = [
        'PLAYER_NAME_stats',     # Redundant name
        'Player_Name',           # Messy salary name
        'merge_key',             # Helper column
        'TEAM_ABBREVIATION',     # Duplicate of 'team'
    ]

    # Redundant "Estimated" (E_) metric set
    e_cols = [col for col in df.columns if col.startswith('E_')]
    cols_to_drop.extend(e_cols)

    # Redundant "sp_work_" metric set
    sp_work_cols = [col for col in df.columns if col.startswith('sp_work_')]
    cols_to_drop.extend(sp_work_cols)
    
    # Redundant "RANK" columns
    rank_cols = [col for col in df.columns if col.endswith('_RANK')]
    cols_to_drop.extend(rank_cols)
    
    # Check for columns that might not exist, just in case
    actual_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
    
    df_cleaned = df.drop(columns=actual_cols_to_drop)
    print(f"✅ Dropped {len(actual_cols_to_drop)} redundant/noise columns.")

    # 3. Rename Columns for Clarity
    cols_to_rename = {
        'PLAYER_NAME_context': 'PLAYER_NAME',
        'Team Name': 'TEAM_NAME',
        'team': 'TEAM_ABBREVIATION',
        'Owner_Net_Worth_Billions': 'OWNER_NET_WORTH_B',
        'Construction_Cost': 'STADIUM_COST',
        'Year_Opened': 'STADIUM_YEAR_OPENED'
    }
    # Only rename columns that actually exist
    actual_cols_to_rename = {k: v for k, v in cols_to_rename.items() if k in df_cleaned.columns}
    df_cleaned = df_cleaned.rename(columns=actual_cols_to_rename)
    print(f"✅ Renamed {len(actual_cols_to_rename)} columns for clarity.")

    # 4. Handle Missing Data
    print("--- Handling Missing Data ---")
    
    # --- Handle Player-Specific Data (Fill with 0 or "Undrafted") ---
    # This logic remains the same.
    fill_zero_cols = ['Followers'] # Only fill followers with 0
    for col in fill_zero_cols:
        if col in df_cleaned.columns:
            df_cleaned[col] = df_cleaned[col].fillna(0)
            
    if 'DRAFT_ROUND' in df_cleaned.columns:
        df_cleaned['DRAFT_ROUND'] = df_cleaned['DRAFT_ROUND'].fillna('Undrafted')
        
    if 'DRAFT_NUMBER' in df_cleaned.columns:
        df_cleaned['DRAFT_NUMBER'] = df_cleaned['DRAFT_NUMBER'].fillna('Undrafted')
    
    print("✅ Filled missing player-specific data.")

    # --- Handle Missing Team Data (User's new logic) ---
    # This is the new, smarter logic you suggested.
    # We copy team data from a teammate (same TEAM_ID) to fill gaps.
    team_cols = [
        'TEAM_ABBREVIATION', 'record', 'active_players', 'avg_team_age',
        'total_cap_used', 'remaining_cap_space', 'active_cap',
        'active_top_3', 'dead_cap', 'TEAM_NAME', 'OWNER_NET_WORTH_B',
        'Stadium_Name', 'Capacity', 'City', 'STADIUM_YEAR_OPENED', 'STADIUM_COST'
    ]
    
    # Filter for columns that actually exist in the dataframe
    existing_team_cols = [col for col in team_cols if col in df_cleaned.columns]
    
    # Group by TEAM_ID and fill NaNs with values from other rows in that same group
    # .ffill() fills forward, .bfill() fills backward to catch all gaps.
    df_cleaned[existing_team_cols] = df_cleaned.groupby('TEAM_ID')[existing_team_cols].ffill().bfill()
    
    print("✅ Smart-filled missing team data by copying from teammates.")

    # --- Handle any remaining NaNs (e.g., a team had NO data) ---
    # After smart-filling, if any NaNs are left (e.g., a whole team was missing),
    # we fill them with 0 or "Unknown" as a final fallback.
    
    # Numeric team cols
    numeric_team_cols = [
        'active_players', 'avg_team_age', 'total_cap_used', 'remaining_cap_space', 
        'active_cap', 'active_top_3', 'dead_cap', 'Capacity', 'STADIUM_COST', 
        'STADIUM_YEAR_OPENED', 'OWNER_NET_WORTH_B'
    ]
    for col in numeric_team_cols:
        if col in df_cleaned.columns:
            df_cleaned[col] = df_cleaned[col].fillna(0)
            
    # Text team cols
    text_team_cols = ['TEAM_ABBREVIATION', 'record', 'TEAM_NAME', 'Stadium_Name', 'City']
    for col in text_team_cols:
        if col in df_cleaned.columns:
            df_cleaned[col] = df_cleaned[col].fillna('Unknown')

    print("✅ Performed final fallback fill for any remaining NaNs.")

    # 5. Save Cleaned File
    output_path = Path(output_filename)
    df_cleaned.to_csv(output_path, index=False)
    
    print(f"\n✅ --- CLEANING COMPLETE --- ✅")
    print(f"Final dataset with {len(df_cleaned)} rows and {len(df_cleaned.columns)} columns saved to:")
    print(f"{output_path.resolve()}")

if __name__ == "__main__":
    clean_master_dataset(SOURCE_FILE, OUTPUT_FILE)

--- Starting Data Cleaning Process for 'master_dataset.csv' ---
✅ Successfully loaded. Found 409 rows and 86 columns.
✅ Dropped 51 redundant/noise columns.
✅ Renamed 6 columns for clarity.
--- Handling Missing Data ---
✅ Filled missing player-specific data.
✅ Smart-filled missing team data by copying from teammates.
✅ Performed final fallback fill for any remaining NaNs.

✅ --- CLEANING COMPLETE --- ✅
Final dataset with 409 rows and 45 columns saved to:
C:\Users\tyler\School\Learn Statistics\STA 160\Project\notebooks\Tyler\master_dataset_cleaned.csv


### Data Dictionary: `master_dataset_cleaned.csv`

---

#### **Group 1: Player Identity & Context**
These columns provide basic, unique information about the player.

* **`PLAYER_ID`**: The unique, stable identifier for each player, assigned by the NBA. This is your best "key" for any player, as names can be similar.
* **`PLAYER_NAME`**: The player's full name, cleaned and standardized (e.g., "LeBron James").
* **`BIRTHDATE`**: The player's date of birth.
* **`COUNTRY`**: The player's home country (e.g., "USA", "France", "Serbia").

---

#### **Group 2: Player Contract & Popularity**
These columns describe the player's professional and financial status.

* **`DRAFT_YEAR`**: The year the player was drafted into the NBA.
* **`DRAFT_ROUND`**: The round they were drafted in. We filled this with **"Undrafted"** if the data was missing.
* **`DRAFT_NUMBER`**: The specific draft pick number (e.g., `1` for the first pick). We also filled this with **"Undrafted"**.
* **`Salary`**: The player's contract salary for the season, cleaned to a single numeric value (e.g., `48728845`).
* **`Followers`**: The player's social media follower count. We filled this with **`0`** if the player was not on the popularity list, assuming they were below the tracking threshold.

---

#### **Group 3: Player On-Court Performance (Advanced Stats)**
These are the advanced metrics from the `Player_Performance_raw.csv` file. We kept the standard set and dropped the redundant `E_` and `sp_work_` versions.

* **`OFF_RATING`**: **Offensive Rating**. An estimate of the points a player produces per 100 possessions.
* **`DEF_RATING`**: **Defensive Rating**. An estimate of the points a player allows per 100 possessions.
* **`NET_RATING`**: **Net Rating**. The difference between Offensive and Defensive Rating (`OFF_RATING` - `DEF_RATING`). This is a key metric for a player's overall on-court impact.
* **`AST_PCT`**: **Assist Percentage**. An estimate of the percentage of teammate field goals a player assisted on while they were on the floor.
* **`AST_TO`**: **Assist to Turnover Ratio**. A simple ratio of how many assists a player has for every one turnover.
* **`AST_RATIO`**: **Assist Ratio**. An estimate of assists per 100 of the player's own possessions.
* **`OREB_PCT`**: **Offensive Rebound Percentage**. An estimate of the percentage of available offensive rebounds a player grabbed.
* **`DREB_PCT`**: **Defensive Rebound Percentage**. An estimate of the percentage of available defensive rebounds a player grabbed.
* **`REB_PCT`**: **Total Rebound Percentage**. An estimate of the percentage of all available rebounds a player grabbed.
* **`TM_TOV_PCT`**: **Turnover Percentage (TOV%)**. An estimate of turnovers committed per 100 plays.
* **`EFG_PCT`**: **Effective Field Goal Percentage**. A shooting percentage that adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal.
* **`TS_PCT`**: **True Shooting Percentage**. A measure of shooting efficiency that accounts for 2-point field goals, 3-point field goals, and free throws. 
* **`USG_PCT`**: **Usage Percentage**. An estimate of the percentage of team plays "used" by a player (i.e., they took a shot, went to the free-throw line, or committed a turnover) while they were on the floor.
* **`PACE`**: **Pace**. An estimate of the number of possessions the player's team has per 48 minutes while that player is on the court.
* **`PACE_PER40`**: **Pace per 40 Minutes**. A normalized version of the Pace metric.
* **`PIE`**: **Player Impact Estimate**. A comprehensive metric that measures a player's overall positive or negative impact on the game, combining many stats.
* **`POSS`**: **Possessions**. The total number of offensive possessions the player was on the court for, used as the denominator for many advanced stats.
* **`FGM_PG`**: **Field Goals Made Per Game**.
* **`FGA_PG`**: **Field Goals Attempted Per Game**.

---

#### **Group 4: Team Identity & Roster Info**
This is the data about the team that *each player belongs to*.

* **`TEAM_ID`**: The unique, stable identifier for each *team* (e.g., `1610612747` for the Lakers). This was our key for the final merge.
* **`TEAM_NAME`**: The full, formal name of the team (e.g., "Los Angeles Lakers").
* **`TEAM_ABBREVIATION`**: The common 3-letter abbreviation (e.g., "LAL").
* **`record`**: The team's win-loss record (e.g., "48-34").
* **`active_players`**: The number of active players on the team's roster.
* **`avg_team_age`**: The average age of the players on the team's roster.

---

#### **Group 5: Team Financials**
This is the detailed salary cap and owner information for the team.

* **`total_cap_used`**: Total salary cap space used by the team, including active players, dead cap, etc.
* **`remaining_cap_space`**: The amount of cap space the team has left under the NBA's salary cap.
* **`active_cap`**: The portion of the cap used only by active, rostered players.
* **`active_top_3`**: The combined salary of the team's top 3 highest-paid players, indicating how "top-heavy" the roster is.
* **`dead_cap`**: Money on the team's salary cap that is being paid to players who are *no longer on the roster*.
* **`OWNER_NET_WORTH_B`**: The estimated net worth of the team's primary owner, in billions of dollars.

---

#### **Group 6: Team Stadium Information**
This is the contextual information about the team's home arena. 

* **`Stadium_Name`**: The official name of the team's home arena (e.g., "Crypto.com Arena").
* **`Capacity`**: The total seating capacity of the stadium for basketball games.
* **`City`**: The city and state where the stadium is located.
* **`STADIUM_YEAR_OPENED`**: The year the stadium first opened.
* **`STADIUM_COST`**: The original construction cost of the stadium, cleaned to a numeric value.

# OK NOW WORK ON DML PULS